In [75]:
import pydeck as pdk
import pandas as pd
import json
import ipywidgets as widgets
import os
import requests
from IPython.display import clear_output

# MAPBOX_API_KEY = "pk.eyJ1IjoidGN5Y2xvbmUiLCJhIjoiY2wwYWx5aW14MG9pdjNjbW8zdmE0dGJkNCJ9.u7D4yFZRHo0PIT45jRuQsA"

BUILDING_LOCATION_DATA = "https://raw.githubusercontent.com/TCyclone/COSC3000/main/Locations.csv"
BUILDING_LOAD_DATA = "https://raw.githubusercontent.com/TCyclone/PowerData/main/"

dfLoad = pd.DataFrame(columns=['Time'])

time = []
start = 2400
for i in range(96):
    if i%4 == 0:
        start = start - 55
        time.append(start)
    else:
        start = start - 15
        time.append(start)
        
dfLoad['Time'] = time

# put branch and node results into dataframe
dfLocation = pd.read_csv(BUILDING_LOCATION_DATA)

files = ['BioEngineeringPV.csv', 'BSB_BuildingPV.csv', 'BSB_LibraryPV.csv', 'CenterHall.csv', 'CSC_BuildingPV.csv', 'CUP_PV.csv', 'EastCampus.csv', 'EBU2_A_PV.csv', 'EBU2_B_PV.csv', 'ElectricShopPV.csv', 'GalbraithHall.csv', 'GarageFleetsPV.csv', 'GeiselLibrary.csv', 'Gilman.csv', 'GilmanParkingPV.csv', 'Hopkins.csv', 'HopkinsParkingPV.csv', 'KeelingA_PV.csv', 'KeelingB_PV.csv', 'KyoceraSkylinePV.csv', 'LeichtagPV.csv', 'Mandeville.csv', 'MayerHallPV.csv', 'MESOM_PV.csv', 'MusicBuilding.csv', 'OslerParkingPV.csv', 'OttersonHall.csv', 'PepperCanyon.csv', 'PoliceDepartment.csv', 'PowellPV.csv', 'PriceCenterA_PV.csv', 'PriceCenterB_PV.csv', 'Processed.csv', 'ProcessedData.csv', 'RadyHall.csv', 'RobinsonHall.csv', 'SDSC_PV.csv', 'SME_SolarPV.csv', 'SocialScience.csv', 'StephenBirchPV.csv', 'StudentServices.csv', 'TradeStreetPV.csv', 'BatteryStorage.csv']
for filename in files:
    power = []
    i = 0
    df = pd.read_csv(os.path.join(BUILDING_LOAD_DATA, filename))
    for i in range(96):
        power.append(df.at[i,'RealPower'])
    dfLoad[filename] = power

powers = []
coloursR = []
coloursG = []
coloursB = []
for i in range(96):
    for j in dfLocation['Location']:
        powers.append(dfLoad.at[i,j])
    dfLocation["time_" + str(dfLoad.at[i,'Time'])] = powers
    powers = []
    
for j in dfLocation['Location']:
    if j[-5].islower():
        coloursR.append(255)
        coloursG.append(0)
    else:
        coloursG.append(255)
        coloursR.append(0)
dfLocation["coloursR"] = coloursR
dfLocation["coloursG"] = coloursG

caption = widgets.Label(value='Initital time')

timeWid = widgets.SelectionSlider(
    options = ['2345', '2330', '2315', '2300', '2245', '2230', '2215', '2200', '2145', '2130', '2115', '2100', '2045', '2030', '2015', '2000', '1945', '1930', '1915', '1900', '1845', '1830', '1815', '1800', '1745', '1730', '1715', '1700', '1645', '1630', '1615', '1600', '1545', '1530', '1515', '1500', '1445', '1430', '1415', '1400', '1345', '1330', '1315', '1300', '1245', '1230', '1215', '1200', '1145', '1130', '1115', '1100', '1045', '1030', '1015', '1000', '945', '930', '915', '900', '845', '830', '815', '800', '745', '730', '715', '700', '645', '630', '615', '600', '545', '530', '515', '500', '445', '430', '415', '400', '345', '330', '315', '300', '245', '230', '215', '200', '145', '130', '115', '100', '45', '30', '15'],
    description='Time:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

def on_drag_end(change):
    clear_output(wait=True)
    display(timeWid, caption)
    newTime = "time_" + str(change['new'])
    times = []
    for i in range(41):
        times.append(dfLocation.at[i,newTime])
    dfLocation["Time"] = times
    caption.value = 'Showing profile at time (in 24hr format): ' + str(change['new'])
    r = make_renderer(dfLocation)
    display(r.show())
#     r.to_html("column_layer.html")

timeWid.observe(on_drag_end, names='value')
display(timeWid, caption)

def make_renderer(dfLocation):
    """Creates the pydeck visualization for rendering"""
    view = pdk.ViewState(latitude=32.87788317, longitude=-117.2372732, bearing=45, pitch=50, zoom=15,)

    view.pitch = 75
    view.bearing = 60

    column_layer = pdk.Layer(
        "ColumnLayer",
        data=dfLocation,
        get_position=["lng", "lat"],
        get_elevation="Time",
        elevation_scale=1,
        radius=20,
        get_fill_color=["coloursR", "coloursG", "0", 200],
        pickable=True,
        auto_highlight=True,
    )

    tooltip = {
        "html": "<b>{Location}</b>, load/generation: <b>{Time} MW</b> ",
        "style": {"background": "grey", "color": "white", "font-family": '"Helvetica Neue", Arial', "z-index": "10000"},
    }
    return pdk.Deck(column_layer, initial_view_state=view, tooltip=tooltip, map_provider="mapbox", map_style=pdk.map_styles.SATELLITE)


r = None

SelectionSlider(continuous_update=False, description='Time:', index=44, options=('2345', '2330', '2315', '2300…

Label(value='Showing profile at time (in 24hr format): 1530')

In [42]:
import pydeck as pdk
import pandas as pd
import json
import ipywidgets as widgets
import os
import requests

BUILDING_LOCATION_DATA = "https://raw.githubusercontent.com/TCyclone/COSC3000/main/LocationsBatt.csv"
BUILDING_LOAD_DATA = "https://raw.githubusercontent.com/TCyclone/PowerData/main/"

dfLoad = pd.DataFrame(columns=['Time'])

time = []
start = 2400
for i in range(96):
    if i%4 == 0:
        start = start - 55
        time.append(start)
    else:
        start = start - 15
        time.append(start)
        
dfLoad['Time'] = time

# put branch and node results into dataframe
dfLocation = pd.read_csv(BUILDING_LOCATION_DATA)

files = ['BioEngineeringPV.csv', 'BSB_BuildingPV.csv', 'BSB_LibraryPV.csv', 'CenterHall.csv', 'CSC_BuildingPV.csv', 'CUP_PV.csv', 'EastCampus.csv', 'EBU2_A_PV.csv', 'EBU2_B_PV.csv', 'ElectricShopPV.csv', 'GalbraithHall.csv', 'GarageFleetsPV.csv', 'GeiselLibrary.csv', 'Gilman.csv', 'GilmanParkingPV.csv', 'Hopkins.csv', 'HopkinsParkingPV.csv', 'KeelingA_PV.csv', 'KeelingB_PV.csv', 'KyoceraSkylinePV.csv', 'LeichtagPV.csv', 'Mandeville.csv', 'MayerHallPV.csv', 'MESOM_PV.csv', 'MusicBuilding.csv', 'OslerParkingPV.csv', 'OttersonHall.csv', 'PepperCanyon.csv', 'PoliceDepartment.csv', 'PowellPV.csv', 'PriceCenterA_PV.csv', 'PriceCenterB_PV.csv', 'Processed.csv', 'ProcessedData.csv', 'RadyHall.csv', 'RobinsonHall.csv', 'SDSC_PV.csv', 'SME_SolarPV.csv', 'SocialScience.csv', 'StephenBirchPV.csv', 'StudentServices.csv', 'TradeStreetPV.csv', 'BatteryStorage.csv', 'NetDemand.csv', 'EastDistribution.csv', 'WestDistribution.csv']
for filename in files:
    power = []
    i = 0
    df = pd.read_csv(os.path.join(BUILDING_LOAD_DATA, filename))
    for i in range(96):
        power.append(df.at[i,'RealPower'])
    dfLoad[filename] = power

powers = []
coloursR = []
coloursG = []
coloursB = []
for i in range(96):
    for j in dfLocation['Location']:
        powers.append(dfLoad.at[i,j])
    dfLocation["time_" + str(dfLoad.at[i,'Time'])] = powers
    powers = []
    
for j in dfLocation['Location']:
    if j[-5].islower():
        coloursR.append(255)
        coloursG.append(0)
    else:
        coloursG.append(255)
        coloursR.append(0)
dfLocation["coloursR"] = coloursR
dfLocation["coloursG"] = coloursG

GREEN_RGB = [0, 255, 0, 200]
RED_RGB = [255, 0, 0, 200]

caption = widgets.Label(value='Initital time')

timeWid = widgets.SelectionSlider(
    options = ['2345', '2330', '2315', '2300', '2245', '2230', '2215', '2200', '2145', '2130', '2115', '2100', '2045', '2030', '2015', '2000', '1945', '1930', '1915', '1900', '1845', '1830', '1815', '1800', '1745', '1730', '1715', '1700', '1645', '1630', '1615', '1600', '1545', '1530', '1515', '1500', '1445', '1430', '1415', '1400', '1345', '1330', '1315', '1300', '1245', '1230', '1215', '1200', '1145', '1130', '1115', '1100', '1045', '1030', '1015', '1000', '945', '930', '915', '900', '845', '830', '815', '800', '745', '730', '715', '700', '645', '630', '615', '600', '545', '530', '515', '500', '445', '430', '415', '400', '345', '330', '315', '300', '245', '230', '215', '200', '145', '130', '115', '100', '45', '30', '15'],
    description='Time:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

def on_drag_end(change, notebook_display = False):
    clear_output(wait=True)
    display(timeWid, caption)
    newTime = "time_" + str(change['new'])
    times = []
    for i in range(44):
        times.append(dfLocation.at[i,newTime])
    dfLocation["Time"] = times
    caption.value = 'Showing profile at time (in 24hr format): ' + str(change['new'])
    r = make_renderer(dfLocation)
    display(r.show())
#     r.to_html("arc_layer.html")
    
timeWid.observe(on_drag_end, names='value')
display(timeWid, caption)

def make_renderer(dfLocation):
    """Creates the pydeck visualization for rendering"""
    view = pdk.ViewState(latitude=32.87788317, longitude=-117.2372732, bearing=45, pitch=50, zoom=15,)

    arc_layer = pdk.Layer(
        "ArcLayer",
        data=dfLocation,
        get_width="Time/12",
        get_source_position=["lngSource", "latSource"],
        get_target_position=["lngTarget", "latTarget"],
        get_tilt=15,
        get_source_color=GREEN_RGB,
        get_target_color=RED_RGB,
        pickable=True,
        auto_highlight=True,
    )

    tooltip = {
        "html": "<b>{Location}</b>, load/generation: <b>{Time} MW</b> ",
        "style": {"background": "grey", "color": "white", "font-family": '"Helvetica Neue", Arial', "z-index": "10000"},
    }
    return pdk.Deck(arc_layer, initial_view_state=view, tooltip=tooltip)


r = None

SelectionSlider(continuous_update=False, description='Time:', index=78, options=('2345', '2330', '2315', '2300…

Label(value='Showing profile at time (in 24hr format): 600')

DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, json_input='{"initialViewState": {"bea…

In [95]:
import pydeck as pdk
import pandas as pd
import json
import ipywidgets as widgets
import os
import requests

BUILDING_LOCATION_DATA = "https://raw.githubusercontent.com/TCyclone/COSC3000/main/LocationsHeat.csv"
BUILDING_LOAD_DATA = "https://raw.githubusercontent.com/TCyclone/PowerData/main/"

dfLoad = pd.DataFrame(columns=['Time'])

time = []
start = 2400
for i in range(96):
    if i%4 == 0:
        start = start - 55
        time.append(start)
    else:
        start = start - 15
        time.append(start)
        
dfLoad['Time'] = time

# put branch and node results into dataframe
dfLocation = pd.read_csv(BUILDING_LOCATION_DATA)

files = ['BatteryStorage.csv', 'ThermalStorage.csv', 'TradeStreetBattery.csv']
for filename in files:
    power = []
    i = 0
    df = pd.read_csv(os.path.join(BUILDING_LOAD_DATA, filename))
    for i in range(96):
        power.append(df.at[i,'RealPower'])
    dfLoad[filename] = power
# display(dfLoad)
# display(dfLocation)
powers = []
coloursR = []
coloursG = []
coloursB = []
for i in range(96):
    for j in dfLocation['Location']:
        powers.append(dfLoad.at[i,j])
    dfLocation["time_" + str(dfLoad.at[i,'Time'])] = powers
    powers = []
    
for j in dfLocation['Location']:
    if j[-5].islower():
        coloursR.append(255)
        coloursG.append(0)
    else:
        coloursG.append(255)
        coloursR.append(0)
dfLocation["coloursR"] = coloursR
dfLocation["coloursG"] = coloursG
display(dfLocation)

RED = [255, 0, 0]
GREEN = [0, 255, 0]


caption = widgets.Label(value='Initital time')

timeWid = widgets.SelectionSlider(
    options = ['2345', '2330', '2315', '2300', '2245', '2230', '2215', '2200', '2145', '2130', '2115', '2100', '2045', '2030', '2015', '2000', '1945', '1930', '1915', '1900', '1845', '1830', '1815', '1800', '1745', '1730', '1715', '1700', '1645', '1630', '1615', '1600', '1545', '1530', '1515', '1500', '1445', '1430', '1415', '1400', '1345', '1330', '1315', '1300', '1245', '1230', '1215', '1200', '1145', '1130', '1115', '1100', '1045', '1030', '1015', '1000', '945', '930', '915', '900', '845', '830', '815', '800', '745', '730', '715', '700', '645', '630', '615', '600', '545', '530', '515', '500', '445', '430', '415', '400', '345', '330', '315', '300', '245', '230', '215', '200', '145', '130', '115', '100', '45', '30', '15'],
    description='Time:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

def on_drag_end(change, notebook_display = False):
    clear_output(wait=True)
    display(timeWid, caption)
    newTime = "time_" + str(change['new'])
    times = []
    colours = []
    for i in range(3):
        times.append(dfLocation.at[i,newTime])
        if (str(dfLocation.at[i,newTime])[0] == "-"):
            colours.append(RED)
        else:
            colours.append(GREEN)
    dfLocation["Time"] = times
    dfLocation["Colour"] = colours
    caption.value = 'Showing profile at time (in 24hr format): ' + str(change['new'])
    r = make_renderer(dfLocation)
    display(r.show())
#     r.to_html("arc_layer.html")
#     r.to_html("heatmap_layer.html")
    
timeWid.observe(on_drag_end, names='value')
display(timeWid, caption)

def make_renderer(dfLocation):
    """Creates the pydeck visualization for rendering"""
    view = pdk.ViewState(latitude=32.87788317, longitude=-117.2372732, bearing=45, pitch=50, zoom=15)
    display(dfLocation)

#     arc_layer = pdk.Layer(
#         "ArcLayer",
#         data=dfLocation,
#         get_width="Time/12",
#         get_source_position=["lngSource", "latSource"],
#         get_target_position=["lngTarget", "latTarget"],
#         get_tilt=15,
#         get_source_color=GREEN_RGB,
#         get_target_color=RED_RGB,
#         pickable=True,
#         auto_highlight=True,
#     )
#     view = pdk.data_utils.compute_view(cattle_df[["lng", "lat"]])
#     view.zoom = 6

    cattle = pdk.Layer(
#         "GeoJsonLayer",
#         data=dfLocation,
#         opacity=0.9,
#         get_position=["lng", "lat"],
#         aggregation=pdk.types.String("MEAN"),
#         color_range="Colour",
#         threshold=1,
#         get_weight="weight",
#         pickable=True,
        "ScatterplotLayer",
        data=dfLocation,
        get_position=["lng", "lat"],
#         get_elevation="Time",
#         elevation_scale=100,
#         radius=50,
#         get_fill_color="Colour",
        pickable=True,
        auto_highlight=True,
        stroked=True,
        filled=True,
        radius_scale=60,
        radius_min_pixels=1,
        radius_max_pixels=100,
        line_width_min_pixels=1,
#         get_position="coordinates",
#         get_radius="exits_radius",
        get_fill_color=[255, 140, 0],
        get_line_color=[0, 0, 0],
    )

    tooltip = {
        "html": "<b>{Location}</b>, load/generation: <b>{Time} MW</b> ",
        "style": {"background": "grey", "color": "white", "font-family": '"Helvetica Neue", Arial', "z-index": "10000"},
    }
    return pdk.Deck(layers=cattle, initial_view_state=view, map_provider="mapbox", map_style=pdk.map_styles.SATELLITE, tooltip=tooltip)


r = None

SelectionSlider(continuous_update=False, description='Time:', index=8, options=('2345', '2330', '2315', '2300'…

Label(value='Initital time')

C:\Users\tomwc\AppData\Local\Temp/ipykernel_17616/937077967.py:86: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dfLocation["Time"] = times
C:\Users\tomwc\AppData\Local\Temp/ipykernel_17616/937077967.py:87: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dfLocation["Colour"] = colours


,Location,lat,lng,time_2345,time_2330,time_2315,time_2300,time_2245,time_2230,time_2215,...,time_115,time_100,time_45,time_30,time_15,time_0,coloursR,coloursG,Time,Colour
0,BatteryStorage.csv,32.880569,-117.240998,-607.088,-4.142,-4.104,-4.047,-4.151,-4.159,-4.101,...,-606.546,-606.697,-606.125,-606.068,-606.476,-606.465,255,0,-4.091,"[255, 0, 0]"
1,ThermalStorage.csv,32.873632,-117.239097,13.61,22.54,13.66,22.15,13.47,17.89,16.18,...,18.59,18.62,27.8,18.82,18.56,18.65,255,0,26.35,"[0, 255, 0]"
2,TradeStreetBattery.csv,32.889469,117.159360,-0.956,-0.953,0.291,0.507,1.176,2.677,0.158,...,3.24,-2.077,5.721,6.064,6.827,4.437,255,0,4.457,"[0, 255, 0]"


DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, json_input='{"initialViewState": {"bea…